In [1]:
import nltk
import utils
import pandas as pd

dream = nltk.corpus.shakespeare.xml("dream.xml")
hamlet = nltk.corpus.shakespeare.xml("hamlet.xml")
macbeth = nltk.corpus.shakespeare.xml("macbeth.xml")
randj = nltk.corpus.shakespeare.xml("r_and_j.xml") 

works = [ 
    {"work_xml": dream,
        "work_name": "Midsummer Night's Dream"}, 
    {"work_xml": hamlet,
        "work_name": "Hamlet"}, 
    {"work_xml": macbeth,
        "work_name": "Macbeth"}, 
    {"work_xml": randj,
        "work_name": "Romeo and Juliet"}
    ]

for title in works:
    main, side = utils.extract_charcs_xml(title["work_xml"], print_charcs=False )
    title["main_charcs"] = main
    title["side_charcs"] = side

    title["parsed_play"] = utils.parse_play_xml(title["work_xml"])

    title["merged"] = utils.merge_play_data(title["parsed_play"], title["main_charcs"], title["side_charcs"])


all_data = []
for w in works:
    xml_tree = w["work_xml"]
    main, side = utils.extract_charcs_xml(xml_tree, print_charcs=False)
    parsed = utils.parse_play_xml(xml_tree)
    merged = utils.merge_play_data(parsed, main, side)
    stats = utils.summarize_play_stats(merged, main, side)
    df = stats["character_df"]  # extract the actual DataFrame
    all_data.append(df)


# --- Combine and export ---
combined_df = pd.concat(all_data, ignore_index=True)
combined_df.to_csv("all_plays_stats.csv", index=False)





A Midsummer Night's Dream
Acts: 5 | Scenes: 9 | Total Speeches: 500 | Total Lines: 2159
Main: 21 | Side: 2 | Ratio: 21:2

Act-level speech totals:
  Act 1: 90 speeches
  Act 2: 62 speeches
  Act 3: 180 speeches
  Act 4: 64 speeches
  Act 5: 104 speeches

Top 10 characters by line count:
                     Play Character  Total Speeches  Total Lines  Scenes Appeared  Acts Appeared  Speech Share (%)  Line Share (%)  Avg Speeches/Scene  Avg Lines/Speech  Verbosity  Talkativeness  Dominance  Focus (Lines/Act)  Breadth (Scene Ratio) Role Type  Acts (Total)  Scenes (Total)  Speeches (Total)  Lines (Total) Main:Side Ratio
A Midsummer Night's Dream   THESEUS              48          233                3              3               9.6           10.79               16.00              4.85       4.85          16.00      10.79              77.67                   0.33      main             5               9               500           2159            21:2
A Midsummer Night's Dream    HELENA  